<a href="https://colab.research.google.com/github/peiyaow/dl790/blob/master/train_dl_model_peiyao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import h5py
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_data():
    train_dataset = h5py.File('/content/drive/My Drive/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
    test_dataset = h5py.File('/content/drive/My Drive/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [0]:
x_train, y_train, x_test, y_test, classes = load_data()

In [0]:
n_train = x_train.shape[0]
n_test = x_test.shape[0]
x_train, x_test = np.array(x_train), np.array(x_test)
y_train, y_test = np.array(y_train), np.array(y_test)
x_train = x_train.reshape((n_train, -1))
x_test = x_test.reshape((n_test, -1))

In [0]:
# using GPU
use_cuda = True
# model parameters
# N: batch size
N, D_in, H1, H2, D_out = 32, x_train.shape[1], 1024, 1024, 1
num_epochs = 500

In [0]:
x_train, x_test = x_train/255, x_test/255

In [0]:
# define prepare dataset
class PrepareData(Dataset):

    def __init__(self, X, y):
        if not torch.is_tensor(X):
            self.X = torch.from_numpy(X)
        if not torch.is_tensor(y):
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [0]:
ds = PrepareData(X=x_train, y=y_train)
ds = DataLoader(ds, batch_size=N, shuffle=True)

In [0]:
# define model
class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
        super(TwoLayerNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(D_in, H1),
            nn.ReLU(),
            nn.Linear(H1, H2),
            nn.ReLU(),
            nn.Linear(H2, D_out),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
      
# dl model
model = TwoLayerNet(D_in, H1, H2, D_out)
#model.cuda()

In [0]:
def lr_lambda(epoch):
    tau = 300
    a_k = min(epoch/tau, 1)
    e_k = (1-a_k) + a_k * 0.01
    return e_k

In [0]:
# define loss function
loss_fn = nn.BCELoss()
lr = 1e-2
optimizer = optim.SGD(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [163]:
for t in range(num_epochs):
    scheduler.step()
    for i, (x, y) in enumerate(ds):
        x = Variable(x).float()
        y = Variable(y).float()
        
        if use_cuda and torch.cuda.is_available():
          x = x.cuda()
          y = y.cuda()
         
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)

        # Compute and print loss
        loss = loss_fn(y_pred.reshape(y.shape), y)
        if i%10 == 0:
          print('[%d, %5d] loss: %.3f' % (t + 1, i + 1, loss))
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

[1,     1] loss: 0.686
[2,     1] loss: 0.606
[3,     1] loss: 0.655
[4,     1] loss: 0.681
[5,     1] loss: 0.669
[6,     1] loss: 0.640
[7,     1] loss: 0.663
[8,     1] loss: 0.608
[9,     1] loss: 0.668
[10,     1] loss: 0.617
[11,     1] loss: 0.555
[12,     1] loss: 0.552
[13,     1] loss: 0.589
[14,     1] loss: 0.629
[15,     1] loss: 0.608
[16,     1] loss: 0.604
[17,     1] loss: 0.647
[18,     1] loss: 0.555
[19,     1] loss: 0.522
[20,     1] loss: 0.541
[21,     1] loss: 0.507
[22,     1] loss: 0.531
[23,     1] loss: 0.622
[24,     1] loss: 0.584
[25,     1] loss: 0.573
[26,     1] loss: 0.587
[27,     1] loss: 0.625
[28,     1] loss: 0.601
[29,     1] loss: 0.578
[30,     1] loss: 0.591
[31,     1] loss: 0.581
[32,     1] loss: 0.618
[33,     1] loss: 0.572
[34,     1] loss: 0.586
[35,     1] loss: 0.473
[36,     1] loss: 0.440
[37,     1] loss: 0.566
[38,     1] loss: 0.490
[39,     1] loss: 0.571
[40,     1] loss: 0.491
[41,     1] loss: 0.437
[42,     1] loss: 0.563
[

In [0]:
ds_test = PrepareData(X=x_test, y=y_test)
ds_test = DataLoader(ds_test)

In [165]:
correct = 0
total = 0
for data in ds_test:
    x, y = data
    x, y = x.float(), y.long()
    outputs = model(x)
    predicted = torch.argmax(torch.cat([1-outputs, outputs], -1))
    total += y.size(0)
    correct += (predicted == y).sum().item()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 74 %
